In [1]:
import numpy as np
import matplotlib.pyplot as plt

#  Internal changing variables 
#  Input intial values 
 
Pot = '-2.0/x'
eps = 1e-4
# The step length
H = 0.001 
H2 = H*H
rad = []
wav = [] 
nr = 0

L = 5
j = 0
# Set an energy reference EG to remove the dimension
# EG --- the ground-state energy
# Establish the function 
Ux = lambda x: eval(Pot)

def Ueff(x):
    global j 
    if x==0: 
        U = 0.0
        return U 
    if j==0:
        U = Ux(x)
        return U
    U = j*(j+1)/x**2 + Ux(x)
    # Potential contains centrifugation potential
    return U

Px = lambda x, A: -float(Ueff(x))+A
# To solve the reduced, second-order differential radial equation:
#      u''(x) + P(x)u(x) = 0

def numerov(a,b,ya,yb,A):
    '''
     ___doc__: applying Numerov's method for solving
               a 2nd ODE (ordinary differential equation).
                a,  b --- starting points
               ya, yb --- function values given to explore the 
                          next point.
               A --- the eigen value for given l & nr
    '''
    global Grid, rad, wav
    if(b>a): # The direction of numerov method
        x1,x2,x3=a,b,b+H
        y1, y2 = ya, yb
    else:
        x3,x2,x1=a,b,b-H
        y2, y3 = yb, ya
    P1,P2,P3 = Px(x1,A), Px(x2,A), Px(x3,A)
    # The recursive relation for the next point
    if(b>a):  #  direction ->
        y3 = (2*(12-5*P2*H2)*y2 - (12+P1*H2)*y1)/(12+P3*H2)
        return y3 
    else:     # direction <-
        y1 = (2*(12-5*P2*H2)*y2 - (12+P3*H2)*y3)/(12+P1*H2)
        return y1
    return  None

# Shooting with the eigenvalue A,
def shoot(A):
    #   __doc__: the MVP subroutine, searching 
    #                for the eigen values of A 
    '''
      Begin to shoot from both sides,
                 using this function and this 
                 function can return the value 
                 of the number of nodes 
    '''
    global rad, wav, L
    L = 5.0
    Grid = int(L/H)
    # initiate the coordinates and wave function
    rad = [i*H for i in range(0,Grid+1)]
    wav = [0 for i in range(0,Grid+1)]
    ns = 0
    # The inital searching interval
  
    while True:
        # intial guess for the beginning point 
        # If potential power higher than r^(-2), 
        # this just returns r^(l+1).  
        cx = rad[1]**2*Px(rad[1],A)
        s = 1.0 + int(0.5+0.5*np.sqrt(1.0-4.0*cx))
        wav[1]= rad[1]**s 
        # inital guess for the ending point 
        X1 = rad[Grid-1]
        X2 = rad[Grid]
        P1 = Px(X1, A) 
        P2 = Px(X2, A)  
        a, b = 1.0, 1.0
        if(P1*P2!=0):
            b = 1.0+int(0.5+0.5*abs(np.log(abs(P1/P2))/np.log(X1/X2)))
            if(b>=eps and b<=10): 
                a = np.sqrt(abs(P2)/X2**(2*b-2))/b 
            elif(b>10):
                a, b = 0.0, 1.0
            else:
                a, b = 1.0, 1.0               
        wav[Grid]=X2**(nr+s)*np.exp(-a*X2**b)
        if(abs(wav[Grid])>eps):
            # Find a proper interval, clear the series 
            rad, wav=[], []
            # Enlarge bounds to the right limit 
            L= L+1.0*b
            Grid = int(L/H)
            # Reinitiate the coordinates and wave function
            rad = [i*H for i in range(0,Grid+1)]
            wav = [0 for i in range(0,Grid+1)]
        else: 
            break             
    wav[0] = 0.0  
    wav[Grid]  =  X2**(s+nr)*np.exp(-a*X2**b)
    wav[Grid-1]=  X1**(s+nr)*np.exp(-a*X1**b)
    # Now begin to shoot from both sides.
    i = 2
    Gh = int(Grid/2)
    while i<= Gh:
        wav[i] = numerov(rad[i-2],rad[i-1],wav[i-2],wav[i-1], A)
        if(wav[i-1]*wav[i]<=0): ns+=1
        if(i==Gh): C = wav[i]
        wav[Grid-i] = numerov(rad[Grid+2-i],rad[Grid+1-i],\
                              wav[Grid+2-i],wav[Grid+1-i], A)
        if(wav[Grid+2-i]*wav[Grid+1-i]<=0): ns+=1           
        i+=1 
    C = C/wav[Gh]
    for i in range(Gh, Grid+1):
        wav[i]=wav[i]*C
    # Test for the continuity across the crucial point :
    # The derivative from two sides    
    wav1 = [0,0,0,0,0,0,0]
    wav2 = [0,0,0,0,0,0,0]
    wav1[0:4]=wav[Gh-3:Gh+1]    # left part of wave function
    wav2[3:7]=wav[Gh:Gh+4]      # right part of wave function 

    i = 0
    for i in range(3):
        wav1[4+i]= numerov(rad[Gh+i],rad[Gh+1+i],wav1[2+i],wav1[3+i], A)
        wav2[2-i]= numerov(rad[Gh-i],rad[Gh-1-i],wav2[4-i],wav2[3-i], A)
    C = deriv3(wav1,H)-deriv3(wav2,H)
    # return the derivative difference
    return ns, C

def deriv3(A3, h): 
    '''
         __doc__: 3-points to obtain 1st order derivative
                  A3 --- input A[0:6] = A[0], A[1], A[2], 
                                              A[3], 
                                        A[4], A[5], A[6].
                   h --- the interval of the grid                    
                   return the result of A[3]
    '''
    deriv=45*(A3[4]-A3[2])-9*(A3[5]-A3[1])+A3[6]-A3[0]
    return deriv/60.0/h
        

# How to bracket the eigen solutions?
def findE(Pot_in='-2.0/x',j_in=0, nr_in=0, eps_in=1e-4, Alow=-1.2, Aupp= 0.0):
    '''
        __doc__:  This subroutine searches the eigen energy 
                  of the ODE (ordinary differential equation)
                  Alow --- The lowest energy guessed, at least 
                           being lower than the ground state.
                  Aupp --- The upper bound energy guessed
                  so that:
                   Alow< A < Aupp
    '''
    #  Input variables 
    global j, Pot, eps,  nr
    Pot, j, eps, nr = Pot_in, j_in, eps_in, nr_in

    while abs(Alow-Aupp)>eps:
        Amid = 0.50*(Alow+Aupp)
        nlow, vlow = shoot(Alow)
        nmid, vmid = shoot(Amid)
        nupp, vupp = shoot(Aupp)
        
        print(Alow, Amid, Aupp)
        print(nlow, nmid, nupp)
        print(vlow, vmid, vupp)
        print()
        if(nlow>nr and  nupp<nr): 
            print('can not bracket the solutions')
            return 
        elif(nr>=nlow and nr<nmid):
            Aupp = Amid 
        elif(nr>nmid and nr<nupp):
            Alow = Amid
        elif(nmid==nr):
            if(vlow*vmid<0 and nmid-nlow==1): 
                Aupp = Amid 
            elif(vmid*vupp<0 and nupp-nmid==1):
                Alow = Amid
            
    
    return  Amid

def main():
    '''
    __doc__:  Plot the profile of the radius versus wavefunction
    '''
    plt.figure(figsize=(5,4),dps=60)
    return 
 

## Harmoic oscillator :

$$
2E_n = (4n_r + 2\ell + 3)\hbar \omega 
$$

## Baryon number density at finite temperature
$$
n_b = \int_0^{k_F}  \frac{k^2dk}{\pi^2} f(\beta)
$$
for given temperature 
$$
f(\beta) = \frac{1}{1+e^{-\beta(\mu - \epsilon_k)}}
$$


In [ ]:
findE(nr_in = 1, j_in=0)
#plt.plot(rad, wav)

In [ ]:
nr = 0
j = 1
Pot='-2.0/x'
print(shoot(-0.075))
plt.plot(rad, wav)

   A  |  n   |   dderiv 
 -----|------|------      
-0.125  |  1   |  -8.11172 
-0.122  |  1   |  -6.35789
-0.120  |  1   |  -4.76677
-0.117  |  1   |  -3.10768
-0.114  |  1   |  -1.48638
-0.112  |  1   |  -0.46336
-0.1115 |  1   |  -0.19803
-0.1111 |  1   |   0.00554
-0.11   |  2   |   0.58609
-0.10   |  2   |   4.99244
-0.09   |  2   |   7.37269
-0.08   |  2   |   6.24500
-0.07   |  2   |   3.05471
-0.06   |  3   |  -0.89798